# Exploring Restaurants in Berlin
#### by André Schlinke
### Abstract

This notebook explores the distribution of different types of restaurants in the center of Berlin. It specifically tries to answer the question where restaurants are located which offer german cuisine. Therefore the concentration of german restaurants are examined for the different boroughs of Berlin. Also, as a last objective, the clustering algorithm k-means is being used to detect culinary hot-spots for opening potential new german restaurants.

## Table of Content

[2. Data](#data)

> [2.1 Data Sources](#data_sources)
>
> [2.2 Data Cleaning](#data_cleaning)

<a id='data'></a>
## 2. Data

To tackle the above depicted problem we need a list of restaurants, their type of cuisine as well as their latitude and longitude. These informations can be gathered by the __Foursquare API__. However, to explore the different venues of a specific area, a radius as well as latitude and longitude values need to be provided by the user. Therefore we take the latitude and longitude of the different postal codes which we get by the __pgeocode__ library. A hand-picked selection of postal codes, their corresponding neighborhoods, as well as their boroughs a provided by a __CSV-file__ which contains data gained by [this](https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Ortsteile_Berlins) wikipedia article.

<a id='data_sources'></a>
### 2.1 Data Sources

In this study a hand-picked selection of the core neighborhoods of Berlin are being analyzed. The data of the different neighborhoods and boroughs were taken from a wikipedia article which you can find [here](https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Ortsteile_Berlins). The data are provided by a CSV-file which contain the different postal codes, neighborhoods and boroughs.

Let's start by importing the several libraries necessary for this notebook.

In [3]:
import numpy as np
import pandas as pd
import pgeocode # library to get latitude and longitude of postal code

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns
sns.set()

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import standard scaler for jointplot
from sklearn.preprocessing import StandardScaler

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


To get a idea of the content of the CSV-file, we read in the file as the DataFrame object __df__ and print the first 5 lines of the DataFrame.

In [4]:
df    = pd.read_csv('berlin_postcode.csv') # Read in CSV-file.
df.head() # Print first 5 lines of DataFrame df.

,Postcode,Neighborhood,Borough
0,10115,Mitte,Mitte
1,10117,Mitte,Mitte
2,10119,Mitte,Mitte
3,10178,Mitte,Mitte
4,10179,Mitte,Mitte


In [5]:
df.shape # Print shape of DataFrame

(97, 3)

In [6]:
df.dtypes # Print DataTypes per column.

Postcode         int64
Neighborhood    object
Borough         object
dtype: object

In [7]:
nomi   = pgeocode.Nominatim('de') # Create Nominatim object for german (de) postal codes.
geo_df = nomi.query_postal_code(df['Postcode'].astype(str).values) # Get latitude and longitude of all postal codes.

In [8]:
geo_df['postal_code'] = geo_df['postal_code'].astype(int)
geo_df.dtypes # Check DataTypes of new  DataFrame.

postal_code         int64
country code       object
place_name         object
state_name         object
state_code         object
county_name        object
county_code         int64
community_name     object
community_code      int64
latitude          float64
longitude         float64
accuracy          float64
dtype: object

In [9]:
geo_df.head() # Print first 5 lines of the DataFrame geo_df.

,postal_code,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,10115,DE,Berlin,Berlin,BE,NaN,0,"Berlin, Stadt",11000,52.5323,13.3846,6.0
1,10117,DE,Berlin,Berlin,BE,NaN,0,"Berlin, Stadt",11000,52.5170,13.3872,6.0
2,10119,DE,Berlin,Berlin,BE,NaN,0,"Berlin, Stadt",11000,52.5305,13.4053,6.0
3,10178,DE,Berlin,Berlin,BE,NaN,0,"Berlin, Stadt",11000,52.5213,13.4096,6.0
4,10179,DE,Berlin,Berlin,BE,NaN,0,"Berlin, Stadt",11000,52.5122,13.4164,6.0


Let's __merge__ the content of the postal codes provided in the DataFrame __df__ with the corresponding latitude and longitude of the DataFrame __geo_df__ into the new DataFrame __berlin_data__.

In [10]:
berlin_data = pd.merge(df, geo_df[['postal_code', 'latitude', 'longitude']], how='left', left_on=['Postcode'], right_on=['postal_code']) # Merge the two DataFrames.

berlin_data.drop(['postal_code'], axis=1, inplace=True) # Drop reduntant column
berlin_data.rename(columns={'latitude':'Latitude', 'longitude':'Longitude'}, inplace=True)
berlin_data.drop_duplicates(inplace=True) # Drop duplicate rows
berlin_data.head()

,Postcode,Neighborhood,Borough,Latitude,Longitude
0,10115,Mitte,Mitte,52.5323,13.3846
1,10117,Mitte,Mitte,52.5170,13.3872
2,10119,Mitte,Mitte,52.5305,13.4053
4,10178,Mitte,Mitte,52.5213,13.4096
6,10179,Mitte,Mitte,52.5122,13.4164


### Coordinates of Berlin, Germany

#### Latitude: 52.5200°, Longitutde: 13.4050°

Information of the latitude and longitude of Berlin were provided by a quick google search.

In [11]:
berlin_latitude, berlin_longitude = 52.5200, 13.4050 # Latitude and longitude of Berlin.

Define function to summarize all neighborhoods to one postal code.

In [12]:
def to_comma_sep_row(df): # Summarizes all neighborhoods with same postal code.
    if df['Neighborhood'].dropna().size == 0:
        df['Neighborhood'] = df['Borough']
    else:
        df['Neighborhood'] = ', '.join(df['Neighborhood'].unique().tolist())
    return df
    
berlin_data = berlin_data.groupby(['Postcode', 'Borough', 'Latitude', 'Longitude']).apply(to_comma_sep_row).drop_duplicates()

Now we create a dictionary containing different colors for each borough.

In [13]:
# colors uses a color map, which will produce an array of colors based on
# the number of labels. We use set(k_means_labels) to get the
# unique labels.
colors_arr   = cm.Paired(np.linspace(0, 1, len(set(berlin_data['Borough']))))

col_dict = {b:colors.to_hex(c) for b, c in zip(set(berlin_data['Borough']), colors_arr)}

### Let's create a map displaying the locations of the different postal codes, as well as the neighborhood they belong to. Each color represents a dfferent borough.

In [14]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[berlin_latitude, berlin_longitude], zoom_start=12)

# add markers to map
for lat, lng, postcode, borough, name in zip(berlin_data['Latitude'], berlin_data['Longitude'], berlin_data['Postcode'], berlin_data['Borough'], berlin_data['Neighborhood']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=col_dict[borough],
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

In [15]:
# @hidden_cell
CLIENT_ID = 'BQL4EG0F0DFM4INHCCZGJ0ONRSQQT4BOYFHGJFC4PLOOY34U' # your Foursquare ID
CLIENT_SECRET = 'KG43YLNKTUGPCCZERULKWGBEALVAA3R21TMQAWIZ2QYSZCS2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Get latitude and longitude of first postal code in DataFrame __berlin_data__.

In [16]:
neighborhood_latitude  = berlin_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = berlin_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name      = berlin_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mitte are 52.5323, 13.3846.


Let's __explore__ the different venues of the first postal code using the __Foursquare API__. Therefore we limit the no. of venues to 100 and explore an area within a radius of 500 meters.

In [17]:
LIMIT = 100 # Limit no. of venues to 100.
neighborhood_radius = 500 # Explore venues within the radius.

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, neighborhood_radius, LIMIT)

In [18]:
results = requests.get(url).json() # Get results of request.

<a id='data_cleaning'></a>
### 2.2 Data Cleaning

The exploration of the venues within a given radius returns a number of different categories of venues. As we are only interested in restaurants for our problem, we only consider categories which contain the string 'Restaurant' in their category. It is clear, that we thereby neglect venues which belong to categories such as 'Fast Food' or 'Food & Drink'. However, as these types do not provide further details on the origin of the respective cuisine style, we consider this neglect as reasonable. Therefore we only focus on restaurants as they generally provide also the local origin of their cuisine in the category name. 

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_restaurants    = nearby_venues.drop(nearby_venues[~nearby_venues['categories'].str.contains('Restaurant')].index) # Drop all venues which do not contain the string 'Restaurant'
nearby_restaurants

,name,categories,lat,lng
9,Cô Chu Vietnamese Gourmet,Vietnamese Restaurant,52.531022,13.384755
14,+84,Vegetarian / Vegan Restaurant,52.532639,13.379511
20,Petrarca,Italian Restaurant,52.529143,13.388316
21,Nordbahnhof Two Buddhas,Asian Restaurant,52.532359,13.384974
22,Schnitzelei,Schnitzel Restaurant,52.528563,13.387228
23,EINS,German Restaurant,52.529460,13.388039
32,Takumi NINE Sapporo,Ramen Restaurant,52.529055,13.384863
44,+84 Asian Deli,Asian Restaurant,52.532419,13.379518


Let's create a first map containing also the restaurants of the first postal code.

In [21]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[berlin_latitude, berlin_longitude], zoom_start=12)

# add neighborhood markers to map
for lat, lng, neighborhood, borough in zip(berlin_data['Latitude'], berlin_data['Longitude'], berlin_data['Neighborhood'], berlin_data['Borough']):
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=col_dict[borough],
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  

# add restaurants markers to map
for lat, lng, name in zip(nearby_restaurants['lat'], nearby_restaurants['lng'], nearby_restaurants['name']):
    label = folium.Popup(name, parse_html=True)
    folium.Marker(
        [lat, lng],
        popup=label).add_to(map_berlin)  
    
map_berlin

Now let's get the restaurants nearby all postal code locations.

In [22]:
def getNearbyRestaurants(postcodes, latitudes, longitudes, boroughs, radius=500): # Gets restaurants per Foursquare API request.
    
    restaurants_list=[]
    for postcode, lat, lng, borough in zip(postcodes, latitudes, longitudes, boroughs):
        print(postcode)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        restaurants_list.append([(
            postcode, 
            lat, 
            lng,
            borough,
            r['venue']['name'], 
            r['venue']['location']['lat'], 
            r['venue']['location']['lng'],  
            r['venue']['categories'][0]['name']) for r in results])

    nearby_restaurants = pd.DataFrame([item for restaurants_list in restaurants_list for item in restaurants_list])
    nearby_restaurants.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude',
                  'Borough',
                  'Restaurant', 
                  'Restaurant Latitude', 
                  'Restaurant Longitude', 
                  'Restaurant Category']
    
    return(nearby_restaurants.drop(nearby_restaurants[~nearby_restaurants['Restaurant Category'].str.contains('Restaurant')].index))

In [23]:
berlin_restaurants = getNearbyRestaurants(postcodes=berlin_data['Postcode'],
                                   latitudes=berlin_data['Latitude'],
                                   longitudes=berlin_data['Longitude'],
                                   boroughs=berlin_data['Borough']
                                  )

10115
10117
10119
10178
10179
10551
10553
10555
10557
10559
13353
10785
10787
13347
13349
13351
13357
13359
13407
13355
13409
10243
10245
10247
10249
10178
10179
10317
10961
10963
10965
10967
10997
10999
10969
10119
10247
10249
10369
10405
10407
10409
10435
10437
10439
13187
13189
10585
10587
10589
10623
10625
10627
10629
14052
14055
14057
14059
10707
10709
10713
10715
10717
10719
10777
14197
10777
10779
10781
10783
10787
10789
10823
10825
10827
10829
12157
12159
12161
12169
12043
12045
12047
12049
12051
12053
12055
12057
12059


In [24]:
berlin_restaurants.head()

,Postcode,Postcode Latitude,Postcode Longitude,Borough,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
9,10115,52.5323,13.3846,Mitte,Cô Chu Vietnamese Gourmet,52.531022,13.384755,Vietnamese Restaurant
14,10115,52.5323,13.3846,Mitte,+84,52.532639,13.379511,Vegetarian / Vegan Restaurant
20,10115,52.5323,13.3846,Mitte,Petrarca,52.529143,13.388316,Italian Restaurant
21,10115,52.5323,13.3846,Mitte,Nordbahnhof Two Buddhas,52.532359,13.384974,Asian Restaurant
22,10115,52.5323,13.3846,Mitte,Schnitzelei,52.528563,13.387228,Schnitzel Restaurant


In [25]:
berlin_restaurants.shape

(1106, 8)

In [26]:
berlin_restaurants = berlin_restaurants[['Borough', 'Restaurant', 'Restaurant Latitude', 'Restaurant Longitude', 'Restaurant Category']]

In order to not consider restaurants multiple times, we drop duplicate listings of restaurants.

In [27]:
berlin_restaurants.reset_index(drop=True, inplace=True)

berlin_restaurants.drop_duplicates(inplace=True) # Drop duplicate restaurants
berlin_restaurants.shape

(1009, 5)